In [99]:
import theano
import theano.tensor as T
import numpy as np
import scipy.io as sio
import sys
import codecs
from collections import Counter
import math
import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu2")
import copy

In [2]:
sys.path.append('/usr0/home/glample/Research/perso/UltraDeep/')

In [3]:
from network import LSTM, FastLSTM
from layer import HiddenLayer, EmbeddingLayer
from learning_method import LearningMethod

In [4]:
path_to_train_src = 'data/train.src'
path_to_train_tgt = 'data/train.tgt'
path_to_dev_src = 'data/dev.src'
path_to_dev_tgt = 'data/dev.tgt'
path_to_test_src = 'data/test.src'

In [82]:
train_src = [line.strip().split() for line in codecs.open(path_to_train_src, 'r', encoding='utf8')]
train_tgt = [line.strip().split() for line in codecs.open(path_to_train_tgt, 'r', encoding='utf8')]
dev_src = [line.strip().split() for line in codecs.open(path_to_dev_src, 'r', encoding='utf8')]
dev_tgt = [line.strip().split() for line in codecs.open(path_to_dev_tgt, 'r', encoding='utf8')]

In [83]:
src_vocab = set()
for line in train_src:
    for word in line:
        src_vocab.add(word)

src_word2ind = {}
src_ind2word = {}

for ind, word in enumerate(src_vocab):
    src_word2ind[word] = ind
    src_ind2word[ind] = word

In [84]:
target_vocab = set()
for line in train_tgt:
    for word in line:
        target_vocab.add(word)

target_word2ind = {}
target_ind2word = {}

for ind, word in enumerate(target_vocab):
    target_word2ind[word] = ind
    target_ind2word[ind] = word

In [85]:
'''
train_src_temp = []
for sentence in train_src:
    train_src_temp.append(np.array([src_word2ind[x] for x in sentence]).astype(np.int32))
train_src_temp = np.array(train_src_temp)
train_src = theano.shared(train_src_temp)

train_tgt_temp = []
for sentence in train_tgt:
    train_tgt_temp.append(np.array([target_word2ind[x] for x in sentence]).astype(np.int32))
train_tgt_temp = np.array(train_tgt_temp)
train_tgt = theano.shared(train_tgt_temp)

dev_src_temp = []
for sentence in dev_src:
    dev_src_temp.append(np.array([src_word2ind[x] for x in sentence]).astype(np.int32))
dev_src_temp = np.array(dev_src_temp)
dev_src = theano.shared(dev_src_temp)

dev_tgt_temp = []
for sentence in dev_tgt:
    dev_tgt_temp.append(np.array([target_word2ind[x] for x in sentence]).astype(np.int32))
dev_tgt_temp = np.array(dev_tgt_temp)
dev_tgt = theano.shared(dev_tgt_temp)
'''

'\ntrain_src_temp = []\nfor sentence in train_src:\n    train_src_temp.append(np.array([src_word2ind[x] for x in sentence]).astype(np.int32))\ntrain_src_temp = np.array(train_src_temp)\ntrain_src = theano.shared(train_src_temp)\n\ntrain_tgt_temp = []\nfor sentence in train_tgt:\n    train_tgt_temp.append(np.array([target_word2ind[x] for x in sentence]).astype(np.int32))\ntrain_tgt_temp = np.array(train_tgt_temp)\ntrain_tgt = theano.shared(train_tgt_temp)\n\ndev_src_temp = []\nfor sentence in dev_src:\n    dev_src_temp.append(np.array([src_word2ind[x] for x in sentence]).astype(np.int32))\ndev_src_temp = np.array(dev_src_temp)\ndev_src = theano.shared(dev_src_temp)\n\ndev_tgt_temp = []\nfor sentence in dev_tgt:\n    dev_tgt_temp.append(np.array([target_word2ind[x] for x in sentence]).astype(np.int32))\ndev_tgt_temp = np.array(dev_tgt_temp)\ndev_tgt = theano.shared(dev_tgt_temp)\n'

In [69]:
src_inp = T.ivector()
tgt_inp = T.ivector()
tgt_op = T.ivector()
index = T.scalar()
#src_lengths = T.ivector()
#tgt_mask = T.fmatrix()

In [70]:
src_embedding_layer = EmbeddingLayer(input_dim=len(src_word2ind), output_dim=64)
tgt_embedding_layer = EmbeddingLayer(input_dim=len(target_word2ind), output_dim=64)
src_lstm_forward = LSTM(input_dim=src_embedding_layer.output_dim, hidden_dim=128, with_batch=False)
src_lstm_backward = LSTM(input_dim=src_embedding_layer.output_dim, hidden_dim=128, with_batch=False)
tgt_lstm = LSTM(input_dim=tgt_embedding_layer.output_dim, hidden_dim=2 * src_lstm_forward.hidden_dim, with_batch=False)
tgt_projection_layer = HiddenLayer(input_dim=tgt_lstm.hidden_dim * 2, output_dim=len(target_word2ind))

In [71]:
src_inp_t = np.random.rand(5,).astype(np.int32)
tgt_inp_t = np.random.rand(5,).astype(np.int32)
tgt_op_t = np.random.rand(5,).astype(np.int32)
#src_lengths_t = np.random.randint(0, 5, size=(10,)).astype(np.int32)
#tgt_mask_t = np.float32(np.random.rand(10, 5).astype(np.float32) > 0.5)

In [72]:
print src_lstm_forward.input_dim, src_lstm_forward.hidden_dim, tgt_lstm.input_dim, tgt_lstm.hidden_dim 

64 128 64 256


In [73]:
'''
# Get embedding matrices
src_emb_inp = src_embedding_layer.link(src_inp)
print 'source embedding', src_emb_inp.eval({src_inp:src_inp_t}).shape
tgt_emb_inp = tgt_embedding_layer.link(tgt_inp)
print 'target embedding', tgt_emb_inp.eval({tgt_inp:tgt_inp_t}).shape

# Get BiLSTM representations
src_lstm_forward.link(src_emb_inp)
src_lstm_backward.link(src_emb_inp[::-1, :])
encoder_representation = T.concatenate((src_lstm_forward.h, src_lstm_backward.h[::-1, :]), axis=1)
print 'src lstm forward', src_lstm_forward.h.eval({src_inp:src_inp_t}).shape
print 'src lstm backward', src_lstm_backward.h.eval({src_inp:src_inp_t}).shape
print 'bilstm', encoder_representation.eval({src_inp:src_inp_t}).shape

# Get Target LSTM representation & Attention Vectors
tgt_lstm.h_0 = encoder_representation[-1]
tgt_lstm.link(tgt_emb_inp)
attention = tgt_lstm.h.dot(encoder_representation.transpose())
attention = attention.dot(encoder_representation)
print 'tgt lstm', tgt_lstm.h.eval({tgt_inp:tgt_inp_t, src_inp:src_inp_t}).shape
print 'attention vectors', attention.eval({tgt_inp:tgt_inp_t, src_inp:src_inp_t}).shape

# Concatenate the attention vectors to the Target LSTM output before predicting the next word
target_representation = T.concatenate([attention, tgt_lstm.h], axis=1)

# Predict each 
proj_output_rep = T.nnet.softmax(tgt_projection_layer.link(target_representation))
print 'proj rep', proj_output_rep.eval({tgt_inp:tgt_inp_t, src_inp:src_inp_t}).shape
'''

"\n# Get embedding matrices\nsrc_emb_inp = src_embedding_layer.link(src_inp)\nprint 'source embedding', src_emb_inp.eval({src_inp:src_inp_t}).shape\ntgt_emb_inp = tgt_embedding_layer.link(tgt_inp)\nprint 'target embedding', tgt_emb_inp.eval({tgt_inp:tgt_inp_t}).shape\n\n# Get BiLSTM representations\nsrc_lstm_forward.link(src_emb_inp)\nsrc_lstm_backward.link(src_emb_inp[::-1, :])\nencoder_representation = T.concatenate((src_lstm_forward.h, src_lstm_backward.h[::-1, :]), axis=1)\nprint 'src lstm forward', src_lstm_forward.h.eval({src_inp:src_inp_t}).shape\nprint 'src lstm backward', src_lstm_backward.h.eval({src_inp:src_inp_t}).shape\nprint 'bilstm', encoder_representation.eval({src_inp:src_inp_t}).shape\n\n# Get Target LSTM representation & Attention Vectors\ntgt_lstm.h_0 = encoder_representation[-1]\ntgt_lstm.link(tgt_emb_inp)\nattention = tgt_lstm.h.dot(encoder_representation.transpose())\nattention = attention.dot(encoder_representation)\nprint 'tgt lstm', tgt_lstm.h.eval({tgt_inp:tg

In [74]:
#
# Model
#
src_emb_dim      = 256  # source word embedding dimension
tgt_emb_dim      = 256  # target word embedding dimension
src_lstm_hid_dim = 512  # source LSTMs hidden dimension
tgt_lstm_hid_dim = 2 * src_lstm_hid_dim  # target LSTM hidden dimension
proj_dim         = 104  # size of the first projection layer
dropout          = 0.5  # dropout rate

n_src = len(src_word2ind)  # number of words in the source language
n_tgt = len(target_word2ind)  # number of words in the target language

# Parameters
params = []

# Source words + target words embeddings layer
src_lookup = EmbeddingLayer(n_src, src_emb_dim, name='src_lookup') # lookup table for source words
tgt_lookup = EmbeddingLayer(n_tgt, tgt_emb_dim, name='tgt_lookup') # lookup table for target words
params += src_lookup.params + tgt_lookup.params

# LSTMs
src_lstm_for = LSTM(src_emb_dim, src_lstm_hid_dim, name='src_lstm_for', with_batch=False)
src_lstm_rev = LSTM(src_emb_dim, src_lstm_hid_dim, name='src_lstm_rev', with_batch=False)
tgt_lstm = LSTM(2 * tgt_emb_dim, tgt_lstm_hid_dim, name='tgt_lstm', with_batch=False)
params += src_lstm_for.params + src_lstm_rev.params + tgt_lstm.params[:-1]

# Projection layers
proj_layer1 = HiddenLayer(tgt_lstm_hid_dim + 2 * src_lstm_hid_dim, n_tgt, name='proj_layer1', activation='softmax')
proj_layer2 = HiddenLayer(2 * src_lstm_hid_dim, tgt_emb_dim, name='proj_layer2', activation='tanh')
params += proj_layer1.params # + proj_layer2.params

In [86]:
is_train_t = 1

src_sentence_t = [3, 4, 2]
tgt_sentence_t = [1, 8, 0, 8, 2]
tgt_gold_t = [1, 3, 2, 2, 1]


# Train status
is_train = T.iscalar('is_train')
# Input sentence
src_sentence = T.ivector()
# Current output translation
tgt_sentence = T.ivector()
# Gold translation
tgt_gold = T.ivector()

src_sentence_emb = src_lookup.link(src_sentence)
tgt_sentence_emb = tgt_lookup.link(tgt_sentence)
print 'src_sentence_emb', src_sentence_emb.eval({src_sentence: src_sentence_t}).shape
print 'tgt_sentence_emb', tgt_sentence_emb.eval({tgt_sentence: tgt_sentence_t}).shape

src_lstm_for.link(src_sentence_emb)
src_lstm_rev.link(src_sentence_emb[::-1, :])

print 'src_lstm_for.h', src_lstm_for.h.eval({src_sentence: src_sentence_t}).shape
print 'src_lstm_rev.h', src_lstm_rev.h.eval({src_sentence: src_sentence_t}).shape

src_context = T.concatenate([src_lstm_for.h, src_lstm_rev.h[::-1, :]], axis=1)
print 'src_context', src_context.eval({src_sentence: src_sentence_t}).shape

tgt_lstm.h_0 = src_context[-1]
repeated_src_context = T.repeat(src_context[-1].dimshuffle('x', 0), tgt_sentence_emb.shape[0], axis=0)
repeated_src_context = proj_layer2.link(repeated_src_context)
print 'repeated src_context', repeated_src_context.eval({src_sentence: src_sentence_t, tgt_sentence: tgt_sentence_t}).shape
tgt_sentence_emb = T.concatenate((tgt_sentence_emb, repeated_src_context), axis=1)
print 'tgt sentence emb', tgt_sentence_emb.eval({src_sentence: src_sentence_t, tgt_sentence: tgt_sentence_t}).shape
tgt_lstm.link(tgt_sentence_emb)
print 'tgt_lstm.h', tgt_lstm.h.eval({src_sentence: src_sentence_t, tgt_sentence: tgt_sentence_t}).shape

transition = tgt_lstm.h.dot(src_context.transpose())
transition = transition.dot(src_context)
print 'transition', transition.eval({src_sentence: src_sentence_t, tgt_sentence: tgt_sentence_t}).shape
# print 'transition_matrix', transition_matrix.eval({src_sentence: src_sentence_t}).shape
# print 'transition_matrix.dot(tgt_lstm.output)', src_context.transpose().dot(src_context.dot(tgt_lstm.output)).eval({src_sentence: src_sentence_t, tgt_sentence: tgt_sentence_t}).shape
# print 'transition_matrix.dot(tgt_lstm.output)', tgt_lstm.h.dot(transition_matrix).eval({src_sentence: src_sentence_t, tgt_sentence: tgt_sentence_t}).shape

transition_last = T.concatenate([transition, tgt_lstm.h], axis=1)
print 'transition_last', transition_last.eval({src_sentence: src_sentence_t, tgt_sentence: tgt_sentence_t}).shape

prediction = proj_layer1.link(transition_last)
print 'prediction', prediction.eval({src_sentence: src_sentence_t, tgt_sentence: tgt_sentence_t}).shape

cost = T.nnet.categorical_crossentropy(prediction, tgt_gold).mean()
print 'cost', cost.eval({src_sentence: src_sentence_t, tgt_sentence: tgt_sentence_t, tgt_gold: tgt_gold_t})

src_sentence_emb (3, 256)
tgt_sentence_emb (5, 256)
src_lstm_for.h (3, 512)
src_lstm_rev.h (3, 512)
src_context (3, 1024)
repeated src_context (5, 256)
tgt sentence emb (5, 512)
tgt_lstm.h (5, 1024)
transition (5, 1024)
transition_last (5, 2048)
prediction (5, 3121)
cost 8.04848766327


In [87]:
'''
cost = - (T.log(proj_output_rep[
    T.arange(tgt_inp.shape[0]).dimshuffle(0, 'x').repeat(tgt_inp.shape[1], axis=1).flatten(),
    T.arange(tgt_inp.shape[1]).dimshuffle('x', 0).repeat(tgt_inp.shape[0], axis=0).flatten(),
    tgt_op.flatten()
]) * tgt_mask.flatten()).sum() / T.neq(tgt_mask, 0).sum()
print cost.eval({tgt_inp:tgt_inp_t, tgt_mask:tgt_mask_t, tgt_op:tgt_op_t, src_inp:src_inp_t, src_lengths:src_lengths_t})
'''

"\ncost = - (T.log(proj_output_rep[\n    T.arange(tgt_inp.shape[0]).dimshuffle(0, 'x').repeat(tgt_inp.shape[1], axis=1).flatten(),\n    T.arange(tgt_inp.shape[1]).dimshuffle('x', 0).repeat(tgt_inp.shape[0], axis=0).flatten(),\n    tgt_op.flatten()\n]) * tgt_mask.flatten()).sum() / T.neq(tgt_mask, 0).sum()\nprint cost.eval({tgt_inp:tgt_inp_t, tgt_mask:tgt_mask_t, tgt_op:tgt_op_t, src_inp:src_inp_t, src_lengths:src_lengths_t})\n"

In [88]:
#cost = T.nnet.categorical_crossentropy(proj_output_rep, tgt_op).mean()
#print cost.eval({tgt_inp:tgt_inp_t, src_inp:src_inp_t, tgt_op:tgt_op_t})

In [89]:
#params = src_embedding_layer.params + tgt_embedding_layer.params + src_lstm_forward.params + src_lstm_backward.params + tgt_lstm.params[:-1] + tgt_projection_layer.params

In [90]:
updates=LearningMethod(clip=5.0).get_updates('adam', cost, params)

In [91]:
f_train = theano.function(
    inputs=[src_sentence, tgt_sentence, tgt_gold],
    outputs=cost,
    updates=updates
)

In [92]:
f_eval = theano.function(
    inputs=[src_sentence, tgt_sentence],
    outputs=prediction,
)

In [93]:
def get_batch(src_sents, tgt_sents, valid=False):
    assert len(src_sents) == len(tgt_sents)
    src_lengths = [len(sent) for sent in src_sents]
    src_max_len = max(src_lengths)
    if valid == False:
        tgt_lengths = [len(sent) for sent in tgt_sents]
        tgt_max_len = max(tgt_lengths)
    return (
        np.array([[src_word2ind[x] for x in sent] + ([0] * (src_max_len - len(sent))) for sent in src_sents]).astype(np.int32),
        np.array(src_lengths).astype(np.int32),
        np.array([[target_word2ind[x] for x in sent[:-1]] + ([0] * (tgt_max_len - len(sent))) for sent in tgt_sents]).astype(np.int32),
        np.array([[target_word2ind[x] for x in sent[1:]] + ([0] * (tgt_max_len - len(sent))) for sent in tgt_sents]).astype(np.int32),
        [([1] * (l - 1)) + ([0] * (tgt_max_len - l)) for l in tgt_lengths]
    )

In [94]:
def get_validation_predictions():
    validation_predictions = []    
    for ind, sent in enumerate(dev_src):
        
        if ind % 300 == 0:
            print ind, len(dev_src)
        src_words = np.array([src_word2ind[x] for x in sent]).astype(np.int32)
        current_outputs = [target_word2ind['<s>']]

        while True:
            next_word = f_eval(src_words, current_outputs).argmax(axis=1)[-1]
            current_outputs.append(next_word)
            #print [target_ind2word[x] for x in current_outputs]
            if next_word == target_word2ind['</s>'] or len(current_outputs) >= 15:
                validation_predictions.append([target_ind2word[x] for x in current_outputs])
                break
    return validation_predictions

In [95]:
def get_test_predictions():
    test_predictions = []    
    for ind, sent in enumerate(test_):
        
        if ind % 300 == 0:
            print ind, len(dev_src)
        src_words = np.array([src_word2ind[x] for x in sent]).astype(np.int32)
        current_outputs = [target_word2ind['<s>']]

        while True:
            next_word = f_eval(src_words, current_outputs).argmax(axis=1)[-1]
            current_outputs.append(next_word)
            #print [target_ind2word[x] for x in current_outputs]
            if next_word == target_word2ind['</s>'] or len(current_outputs) >= 15:
                validation_predictions.append([target_ind2word[x] for x in current_outputs])
                break
    return validation_predictions

In [ ]:
f = open('blue_valid_log.txt', 'w')
all_costs = []
batch_size = 50
n_epochs = 100
best_valid_preds = None
best_valid_score = -sys.maxint
for i in xrange(n_epochs):
    print 'Starting epoch %i' % i
    indices = range(len(train_src))
    np.random.shuffle(indices)
    train_src_batch = [train_src[ind] for ind in indices]
    train_tgt_batch = [train_tgt[ind] for ind in indices]
    assert len(train_src_batch) == len(train_tgt_batch)
    costs = []
    for j in xrange(len(train_src_batch)):
        #s_sent, s_length, t_inp, t_op, mask = get_batch(train_src_batch[j:j + batch_size], train_tgt_batch[j:j+batch_size])
        new_cost = f_train(
            np.array([src_word2ind[x] for x in train_src_batch[j]]).astype(np.int32),
            np.array([target_word2ind[x] for x in train_tgt_batch[j]][:-1]).astype(np.int32),
            np.array([target_word2ind[x] for x in train_tgt_batch[j]][1:]).astype(np.int32),
        )
        all_costs.append((j, new_cost))
        costs.append(new_cost)
        if j % 300 == 0:
            print j, np.mean(costs)
            costs = []
        if np.isnan(new_cost):
            print 'NaN detected.'
            break
        if j % 10000 == 0:
            valid_preds = get_validation_predictions()
            print '==================================================================='
            print 'Epoch %i BLEU on Validation : %s ' % (i, get_validation_bleu(valid_preds))
            print '==================================================================='
            if float(get_validation_bleu(valid_preds)) >= best_valid_score:
                best_valid_score = float(get_validation_bleu(valid_preds))
                best_valid_preds = copy.deepcopy(valid_preds)
                print 'Found new best validation score %f ' % (best_valid_score)
            f.write('Epoch %d Minibatch %d BLEU on Validation : %s \n' % (i, j, get_validation_bleu(valid_preds)))

    if np.isnan(new_cost):
        print 'NaN detected.'
        break
    valid_preds = get_validation_predictions()
    print '==================================================================='
    print 'Epoch %i BLEU on Validation : %s ' % (i, get_validation_bleu(valid_preds))
    print '==================================================================='
f.close()

Starting epoch 0
0 0.633995
0 1006
300 1006
600 1006
900

In [114]:
best_valid_score

27.91

In [238]:
test_batch_size = 100

for i in xrange(0, len(dev_src), test_batch_size):
    src_lengths = np.array([len(x) for x in dev_src[i:i+test_batch_size]]).astype(np.int32)
    src_maxlen = np.max(src_lengths)
    src_inps = np.array([ [src_word2ind[x] for x in dev_src[j]] + ([0] * (src_maxlen - len(dev_src[j]))) for j in xrange(i, i + test_batch_size)]).astype(np.int32)
    current_outputs = [[target_word2ind['<s>']] for _ in xrange(len(features))]
    final_outputs = [None] * len(features)

    mapping = {j: j for j in xrange(len(features))}

    while len(current_outputs) > 0:
        to_delete = []
        next_words = f_eval(src_inps, src_lengths, current_outputs)[:, -1, :].argmax(axis=1)
        assert len(mapping) == len(next_words) == len(current_outputs)
        for j in xrange(len(next_words)):
            current_outputs[j].append(next_words[j])
            if next_words[j] == target_word2ind['</s>'] or len(current_outputs[j]) >= 20:
                final_outputs[mapping[j]] = current_outputs[j]
                to_delete.append(j)
        for j in sorted(to_delete)[::-1]:
            del features[j]
            del current_outputs[j]
            del mapping[j]
        new_index = 0
        for k in sorted(mapping.keys()):
            if k > new_index:
                mapping[new_index] = mapping[k]
                del mapping[k]
            new_index += 1

    assert all(final_outputs)
    break

In [40]:
xx = f_eval(np.array([src_word2ind[x] for x in dev_src[5]]).astype(np.int32), np.array([target_word2ind[x] for x in dev_tgt[5]][:-1]).astype(np.int32)).argmax(axis=1)

In [41]:
[target_ind2word[_] for _ in xx]

[u'how',
 u'do',
 u'i',
 u'use',
 u'the',
 u'safe-',
 u'restaurant',
 u'box',
 u'?',
 u'</s>']

In [42]:
dev_tgt[5]

[u'<s>',
 u'how',
 u'do',
 u'i',
 u'use',
 u'a',
 u'safe',
 u'deposit',
 u'box',
 u'?',
 u'</s>']

In [97]:
valid_preds = get_validation_predictions()

0 1006
300 1006
600 1006
900 1006


In [98]:
get_validation_bleu(valid_preds)

'28.21'

In [45]:
for x in valid_preds[:10]:
    print ' '.join(x)

currency my throbbing shower is <unk> . </s> nine four five . </s> all the
currency a lot of august fifteenth . </s> number five . </s> one of them
currency for me to japan ? </s> over the stairs to japan . </s> .
currency one hundred dollars in a night . </s> . </s> </s> </s> </s> <unk>
currency on a newspaper ? </s> up with a newspaper . </s> </s> <unk> .
currency for <unk> ? </s> </s> three ? </s> </s> <unk> ? </s> </s> <unk>
currency where where the boarding is where ? </s> the thirty- four ? </s> </s>
currency a table is free to go . </s> if it 's available ? </s>
currency these are <unk> over . </s> </s> will be <unk> . </s> </s> <unk>
currency be an hour . </s> 's <unk> . </s> ? </s> </s> <unk> .


In [37]:
def bleu_stats(hypothesis, reference):
    stats = []
    stats.append(len(hypothesis))
    stats.append(len(reference))
    for n in xrange(1,5):
        s_ngrams = Counter([tuple(hypothesis[i:i+n]) for i in xrange(len(hypothesis)+1-n)])
        r_ngrams = Counter([tuple(reference[i:i+n]) for i in xrange(len(reference)+1-n)])
        stats.append(max([sum((s_ngrams & r_ngrams).values()), 0]))
        stats.append(max([len(hypothesis)+1-n, 0]))
    return stats

In [38]:
def bleu(stats):
    if len(filter(lambda x: x==0, stats)) > 0:
        return 0
    (c, r) = stats[:2]
    log_bleu_prec = sum([math.log(float(x)/y) for x,y in zip(stats[2::2],stats[3::2])]) / 4.
    return math.exp(min([0, 1-float(r)/c]) + log_bleu_prec)

In [39]:
def get_validation_bleu(hypotheses):
    stats = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
    ref_lines = [line.strip().split() for line in open(path_to_dev_tgt, 'r')]
    for hyp, ref in zip(hypotheses, ref_lines):
        stats += np.array(bleu_stats(hyp, ref))
    return "%.2f" % (100*bleu(stats))